In [ ]:
#General libraries
import os, argparse
import pickle
#from sklearn.decomposition import PCA
import numpy as np

#Torch libraries
import torch 
from torch import nn

#Custom libraries
#from load_datasets import load_dataset, create_dataloaders
from utils import Train_val_split, Dynamics_Dataset, Test_Dynamics_Dataset
from utils import fix_random_seeds,to_np
#from source.ide_func import NNIDEF, NeuralIDE
#from IE_source.solver import IESolver_monoidal
import IE_source.kernels as kernels
from IE_source.experiments import Full_experiment_AttentionalIE_PDE
from torch.utils.data import SubsetRandomSampler
from IE_source.kernels import model_blocks

from IE_source.Attentional_IE_solver import masking_function, Integral_attention_solver

if torch.cuda.is_available():  
    device = "cuda:0" 
else:  
    device = "cpu"
    

parser = argparse.ArgumentParser(description='Neural IE')
parser.add_argument('-root_path', metavar='DIR', default='',
                    help='path to dataset')
parser.add_argument('-dataset-name', default='stl10',
                    help='dataset name', choices=['acrobot_dataset'])

parser.add_argument('-j', '--workers', default=12, type=int, metavar='N',
                    help='number of data loading workers (default: 32)')
parser.add_argument('--epochs', default=3000, type=int, metavar='N',
                    help='number of total epochs to run')
parser.add_argument('-b', '--batch_size', default=20, type=int,
                    metavar='N',
                    help='mini-batch size (default: 256), this is the total '
                         'batch size of all GPUs on the current node when '
                         'using Data Parallel or Distributed Data Parallel')
parser.add_argument('--lr', '--learning-rate', default=1e-4, type=float,
                    metavar='LR', help='initial learning rate', dest='lr')
parser.add_argument('--wd', '--weight-decay', default=1e-4, type=float,
                    metavar='W', help='weight decay (default: 1e-4)',
                    dest='weight_decay')
parser.add_argument('--seed', default=None, type=int,
                    help='seed for initializing training. ')
parser.add_argument('--disable-cuda', action='store_true',
                    help='Disable CUDA')
parser.add_argument('--fp16-precision', action='store_true',
                    help='Whether or not to use 16-bit precision GPU training.')

parser.add_argument('--out_dim', default=128, type=int,
                    help='feature dimension (default: 128)')
parser.add_argument('--log-every-n-steps', default=100, type=int,
                    help='Log every n steps')
parser.add_argument('--temperature', default=0.07, type=float,
                    help='softmax temperature (default: 0.07)')
parser.add_argument('--n-views', default=2, type=int, metavar='N',
                    help='Number of views for contrastive learning training.')
parser.add_argument('--gpu-index', default=0, type=int, help='Gpu index.')
parser.add_argument('--model', default='simclr', choices=['simclr','lipschitz_simclr','vae','gan'], 
                    help='Models to be used')
parser.add_argument('--mode', default='train', choices=['train','evaluate'], 
                    help='Set to ''evaluate'' if inference is desired')
parser.add_argument('--training_split', default=0.25,type=float, 
                    help='Fraction of the samples that will be used for validation')
parser.add_argument('--resume_from_checkpoint', default=None, 
                    help='Give string to run number. Ex: "run12"')
parser.add_argument('--plot_freq', default=1, type=int,help='')
parser.add_argument('--experiment_name', default=None,help='')


In [ ]:
args = parser.parse_args("")
args.model='nie'
args.mode='train'
args.mode = 'evaluate'
args.dataset_name = 'integral_equations'
args.seed = 7
args.experiment_name = 'Burgers_test-longT-4'
args.plot_freq = 10
args.device = device
args.num_dim_plot = 2
args.lr = 1e-3
args.min_lr=1e-7
args.T_max = 51
args.plat_patience = 10
args.factor = 0.5
args.lr_scheduler = 'CosineAnnealingLR'
args.resume_from_checkpoint = 'run36'
fix_random_seeds(args.seed)
args.perturbation_to_obs0=None
args.training_split=0.2
args.smoothing_factor=.5

In [ ]:
args.kernel_split = True
args.free_func_nn = False
args.kernel_type_nn = True
args.G_NN = True
args.num_internal_points = 100 
args.plot_F_func = False
args.f_nn = False
args.max_iterations=3
args.sampling_points=100 
args.time_points=2  
args.support_tensors=False
args.support_test=False
args.test_points=50
args.combine_points=False
args.fourier_transform = False
args.linear_decoder = False
args.plot_as_image = True
args.plot_eval = False

In [ ]:
args.n_batch=32

In [ ]:
args.dim = 4
args.dim_emb=8
args.n_head=4
args.n_blocks=6
args.n_ff=64
args.attention_type='galerkin'
args.final_block=False

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import scipy.io as spio

In [ ]:
import mat73

# Burgers dataset has shape explained in https://github.com/zongyi-li/fourier_neural_operator. 1k samples for grid with size of 8192.

# Navier-Stokes dataset same link as above

In [ ]:
Eqn_type = 'Burgers'
burgers_t=25
args.burgers_t=burgers_t


In [ ]:
if Eqn_type == 'Burgers':
    if args.mode=='train':
        Data = torch.load('Burgers_1k_t400')
    else:
        print('loading test set')
        Data = torch.load('Burgers_Data_N200_t400')
else:
    Data = mat73.loadmat('Navier-Stokes_V1e-3_N5000_T50.mat')


In [ ]:
Data

In [ ]:
if Eqn_type == 'Navier-Stokes' or burgers_t==2:
    Data['a'].shape

In [ ]:
if Eqn_type == 'Burgers' and burgers_t==2:
    Data['a_smooth'].shape

In [ ]:
if Eqn_type == 'Burgers' and burgers_t==2:
    Data['a_smooth_x'].shape

In [ ]:
if Eqn_type == 'Navier-Stokes' or burgers_t==2:
    Data['u'].shape

In [ ]:
if Eqn_type == 'Burgers' and burgers_t>2:
     Data.shape

In [ ]:
if Eqn_type == 'Navier-Stokes' or burgers_t==2:
    Data_u = torch.from_numpy(Data['u'])

In [ ]:
if Eqn_type == 'Navier-Stokes' or burgers_t==2:
    Data_a = torch.from_numpy(Data['a'])

In [ ]:
if Eqn_type == 'Navier-Stokes' or burgers_t==2:
    Data_u.shape

In [ ]:
if Eqn_type == 'Navier-Stokes' or burgers_t==2:
    Data_a.shape

In [ ]:
if Eqn_type == 'Burgers' and burgers_t>2:
    Data.shape

In [ ]:
t_max = 1 
t_min = 0
n_points = 512

index_np = np.arange(0, n_points, 1, dtype=int)
index_np = np.hstack(index_np[:, None])
times_np = np.linspace(t_min, t_max, num=n_points)
times_np = np.hstack([times_np[:, None]])

###########################################################
times = torch.from_numpy(times_np[:, :, None]).to(device)
times = times.flatten().float()
###########################################################
args.time_interval=t_min, t_max

In [ ]:
#Plot some of the data to visualize it
if Eqn_type == 'Burgers' and burgers_t==2:
    for i in range(10):
        plt.plot(torch.linspace(0,1,8192),Data_a[i,:])
        plt.plot(torch.linspace(0,1,8192),Data_u[i,:])
elif Eqn_type == 'Burgers' and burgers_t>2:
    for i in range(5):
        plt.plot(torch.linspace(0,1,Data.shape[1]),Data[i,:,1])
        plt.plot(torch.linspace(0,1,Data.shape[1]),Data[i,:,-1])
else:
    for i in range(10):
        plt.plot(torch.linspace(0,1,50),Data_u[i,17,17,:])

In [ ]:
from torchcubicspline import(natural_cubic_spline_coeffs, 
                             NaturalCubicSpline)

In [ ]:
#visualize few times
if Eqn_type == 'Burgers' and burgers_t>2:
    for i in range(1,100,10):
        #plt.figure(i)
        plt.plot(torch.linspace(0,1,Data.shape[1]),Data[0,:,i],label='Data'+str(i))
        #plt.legend()

In [ ]:
Data.shape

In [ ]:
if burgers_t>2:
    Data = Data[:,:,:400]

In [ ]:
if Eqn_type == 'Burgers' and burgers_t==2:
    for i in range(1024):
        Data_i0 = Data_a[i:i+1,:].unsqueeze(-1)
        Data_i1 = Data_u[i:i+1,:].unsqueeze(-1)
        if i == 0:
            Data = torch.cat([Data_i0,Data_i1],-1)
        else:
            Data = torch.cat([Data,torch.cat([Data_i0,Data_i1],-1)])
        
    
elif Eqn_type == 'Burgers' and burgers_t>2:
    pass
else:
    Data = Data_u[:10,:,:,:]

In [ ]:
Data.shape

In [ ]:
Data = Data.to(device)

In [ ]:
Data = Data.float()

In [ ]:
ids = np.tile(np.linspace(0,Data.shape[1]-1,num=n_points, dtype=np.int64),(Data.shape[1],1))

In [ ]:
ids[0]

In [ ]:
if args.mode=='train': 
    if burgers_t>2:
        t_ids = np.tile(np.linspace(1,Data.shape[-1]-1,num=burgers_t, dtype=np.int64),(Data.shape[1],1))
        print(t_ids[0])
else:
    t_ids = np.tile(np.linspace(1,int(Data.shape[-1]/4),num=burgers_t, dtype=np.int64),(Data.shape[1],1))
    print(t_ids[0])

In [ ]:
Data = Data[:,ids[0],:]

In [ ]:
Data.shape

In [ ]:
if burgers_t>2:
    Data = Data[:,:,t_ids[0]]

In [ ]:
Data.shape

In [ ]:
if burgers_t>2:
    args.time_points=burgers_t

In [ ]:
Data.shape

In [ ]:
ts_integration = torch.linspace(0,1,400)[t_ids[0]]
print(ts_integration)
args.ts_integration = ts_integration

In [ ]:

n_steps = 5000
print('Data.shape: ',Data.shape)

Data_splitting_indices = Train_val_split(np.copy(index_np),0)
Train_Data_indices = Data_splitting_indices.train_IDs()
Val_Data_indices = Data_splitting_indices.val_IDs()
print('\nlen(Train_Data_indices): ',len(Train_Data_indices))
print('Train_Data_indices: ',Train_Data_indices)
print('\nlen(Val_Data_indices): ',len(Val_Data_indices))
print('Val_Data_indices: ',Val_Data_indices)
Dataset = Dynamics_Dataset(Data,times)

Dataset_all = Test_Dynamics_Dataset(Data,times)

# For the sampler
train_sampler = SubsetRandomSampler(Train_Data_indices)
valid_sampler = SubsetRandomSampler(Val_Data_indices)
    

dataloaders = {'train': torch.utils.data.DataLoader(Dataset, sampler=train_sampler, batch_size = args.batch_size, drop_last=True),
               'val': torch.utils.data.DataLoader(Dataset, sampler=valid_sampler, batch_size = args.batch_size, drop_last=True),
               'test': torch.utils.data.DataLoader(Dataset_all, batch_size = len(np.copy(index_np))),
              }

model = model_blocks(args.dim+1,
                     args.dim_emb,
                     args.n_head,
                     args.n_blocks,
                     args.n_ff,
                     args.attention_type,
                     args.final_block,
                     dropout=0.3)

if torch.cuda.is_available():
    model = model.cuda()

In [ ]:
args.range_imshow = np.array([np.quantile(to_np(Data).flatten(), 0.05), np.quantile(to_np(Data).flatten(), 0.95)])

In [ ]:
args.range_imshow

Data.size()

In [ ]:
exp_mode = 'Fredholm'

In [ ]:
#Fredholm mode
mask = None

In [ ]:
#Volterra mode
if exp_mode == 'Volterra':
    masking_map =  masking_function(lambda x: 0.,lambda x: x,n_batch=1)
    mask_times = times
    mask = masking_map.create_mask(mask_times)

In [ ]:
print(mask)

In [ ]:
args.n_patch = 64

In [ ]:
Encoder = kernels.ConvNeuralNet1D(
                            1,
                            args.dim,
                            args.dim,
                            hidden_ff=32,
                            Data_shape1=512,
                            n_patch=args.n_patch
).to(args.device)

In [ ]:
args.shapes = Encoder(Data[:4,:,:1].permute(0,2,1)).shape

In [ ]:
args.n_points = args.shapes[-1]

In [ ]:
class Decoder_NN(nn.Module):
    def __init__(self,in_dim,out_dim,shapes,NL=nn.ELU):
        super(Decoder_NN, self).__init__()
        self.in_dim = in_dim
        self.out_dim = out_dim
        self.n_layers = len(shapes) - 1
        self.shapes = shapes
        self.first = nn.Linear(in_dim,shapes[0])
        self.layers = nn.ModuleList([nn.Linear(shapes[i],shapes[i+1]) for i in range(self.n_layers)])
        self.last = nn.Linear(shapes[-1], out_dim)
        self.NL = NL(inplace=True) 
        
    def forward(self, y):
        y_in = y.permute(0,2,1,3)
        y_in = y_in.flatten(2,3)
        y = self.NL(self.first.forward(y_in))
        for layer in self.layers:
            y = self.NL(layer.forward(y))   
        y_out = self.last.forward(y)
        y = y_out.permute(0,2,1)

        return y

In [ ]:
Decoder = Decoder_NN(args.shapes[-1]*args.dim,n_points,[64,128]).to(args.device)

In [ ]:
kernels.flatten_kernel_parameters(model).shape[0]+kernels.flatten_kernel_parameters(Encoder).shape[0]+kernels.flatten_kernel_parameters(Decoder).shape[0]



In [ ]:
args.epochs=5000

In [ ]:
args.print_ts=False
if args.print_ts is True:
    args.freq_print_ts = int(args.epochs/500)*args.plot_freq

In [ ]:
Full_experiment_AttentionalIE_PDE(model,Encoder,Decoder,Data, times, index_np, mask, times, args, extrapolation_points=None)